## **Estructura de sub-etapas del procesamiento del habla:**

---

```
├── Input Layer
│   ├── Audio Capture
│   └── ASR (Speech-to-Text)
├── NLP Layer
│   ├── Text Preprocessing
│   ├── Intent Detection
│   ├── Entity Recognition
│   └── Sentiment Analysis
├── Dialogue Management
│   ├── State Management
│   ├── Response Generation
│   ├── Response Selection
│   └── Context Handling
├── Output Layer
│   ├── TTS (Text-to-Speech)
│   └── Audio Playback

```


### **Estructura operativa para cada sub-etapa:**

**Selección de técnicas de modelado:** En este caso se utilizara [] para la etapa de reconocimiento de voz, [] para el procesamiento y analisis de texto y [] para la conversion de texto a voz.

**Generación de un diseño de comprobación:** Para elegir el modelo correcto, este devera ser el que pondere mas alto en un promedio de las diferentes metricas de evaluacion del modelo.

**Generación de los modelos:** Se definiran y configuraran los parametros del modelo para pasar a la etapa de ejecucion y descripcion.

**Evaluación del modelo:** Se evaluaran los diferentes modelos de manera individual en busqueda de optimizar sus parametros y escoger la combinacion mas optima entre modelos / parametros individuales.

In [ ]:
# Execute if use colab and you need import files
from google.colab import files

In [ ]:
# Execute if use colab and you need import files
drive.mount('/content/drive')

In [1]:
# Execute if use colab
#%%bash
!apt-get update
!apt-get install python3
!apt-get install -venv
!python3 -m venv proc_habla
!source proc_habla/bin/activate

zsh:1: command not found: apt-get
zsh:1: command not found: apt-get
zsh:1: command not found: apt-get


In [2]:
# Create and activate virtual env for speech processing stage
!python3 -m venv proc_habla
!source proc_habla/bin/activate

## **1. Entrada de Usuario (Reconocimiento del Habla)**

### Librerias y paquetes

In [3]:
!pip3 install SpeechRecognition
!pip3 install sounddevice
!pip3 install scipy
!pip3 install pydub
!pip install pocketsphinx

In [4]:
import speech_recognition as sr
import numpy as np
import wave
import pandas as pd
import sounddevice as sd
from scipy.io.wavfile import write
import queue
import threading
from pydub import AudioSegment
import subprocess
import zipfile
import os
import pandas as pd

In [5]:
# Ruta del archivo zip descargado
ruta_modelo_comprimido_pocketsphinx_esp = "../datos/brutos/modelos_proc_habla/modelo_esp.zip"

# Directorio donde se colocarán los archivos descomprimidos
ruta_modelo_descomprimido_pocketsphinx_esp = "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/speech_recognition/pocketsphinx-data/es-ES/"

# Descomprimir el archivo zip
with zipfile.ZipFile(ruta_modelo_comprimido_pocketsphinx_esp, 'r') as zip_ref:
    zip_ref.extractall(ruta_modelo_descomprimido_pocketsphinx_esp)

print("Archivos descomprimidos en:", ruta_modelo_descomprimido_pocketsphinx_esp)

Archivos descomprimidos en: /Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/speech_recognition/pocketsphinx-data/es-ES/


In [6]:
try:
    subprocess.run(["sudo", "apt-get", "install", "ffmpeg"])
    print("FFmpeg instalado exitosamente en sistemas basados en Debian.")
except Exception as e:
    print("Error al instalar FFmpeg:", e)
try:
    subprocess.run(["brew", "install", "ffmpeg"])
    print("FFmpeg instalado exitosamente en macOS utilizando Homebrew.")
except Exception as e:
    print("Error al instalar FFmpeg:", e)

sudo: a terminal is required to read the password; either use the -S option to read from standard input or configure an askpass helper
sudo: a password is required


FFmpeg instalado exitosamente en sistemas basados en Debian.
FFmpeg instalado exitosamente en macOS utilizando Homebrew.


To reinstall 7.0.1, run:
  brew reinstall ffmpeg


### DataFrames

In [ ]:
columns_consultas_df = ['id_consulta', 'id_cliente', 'formato_consulta', 'transcripción_audio', 'entrada_texto', 'productos_detectados']

In [ ]:
consultas_df = pd.DataFrame(columns=columns_consultas_df)

In [9]:
path_csv_rutas_verduras = '../datos/procesaodos/VerdurasporSupermercado.csv'
df_frutas_verduras = pd.read_csv(path_csv_rutas_verduras)

In [10]:
df_frutas_verduras.head(10)

,Supermercado,Producto,Precio
0,Carrefour,Ajo suelto x kg.,"$ 7.690,00"
1,Carrefour,Tomate perita x kg.,"$ 5.499,00"
2,Carrefour,Sandía x kg.,"$ 2.499,00"
3,Carrefour,Rúcula x 1 atado,"$ 699,00"
4,Carrefour,Acelga x paquete,"$ 1.189,00"
5,Carrefour,Cebolla morada x kg.,"$ 1.799,00"
6,Carrefour,Lechuga morada kg.,"$ 5.599,00"
7,Carrefour,Lechuga francesa hidropónica 220 g.,"$ 1.899,00"
8,Carrefour,Lima taití x kg.,"$ 2.999,00"
9,Carrefour,Maní tostado pelado sin sal x kg.,"$ 5.290,00"


## **1. Módulo de Captura de entradas por texto y audio:**

### a. Captura de entradas de texto

In [ ]:
#Conexion con api de fast-api
# aui iria la conexion con un script encargado de conectar con la api de fast-api

In [11]:
texto_prueba = 'estoy buscando tomates en oferta pero que esten bien rojitos :p'

### b. Captura de Audio

*Bugs*

- [ ] verificar existencia de id_cliente, id_consulta y diccionario con respuesta dentro de dataframe ouput
- [ ] Definir como se llamara a la funcion chatbot
    - [ ] Definir el anidamiento de funciones, para que con solo llamar a chatbot, me entregue el resultado (lista de productos limpia)
- [ ] Contar considencias de lemas consulta usuario por lemas en columnas productos_tokens_lemmas
    - [ ] almacenar en un diccionario
    - [ ] ordenrar diccionario por orden de mayor a menor considencia
- [ ] text to speech
- [ ] reconocimiento de voz

In [12]:
SAMPLE_RATE = 44100  # Tasa de muestreo
DURATION = 10  # Duración de la grabación en segundos
AUDIO_FILES_PATH = "../datos/brutos/audios_proc_habla/"
FILENAME = "output"  # Nombre del archivo de salida
EXTENCION_ENTRADA = '.wav'
EXTENCION_SALIDA = '.aiff'

In [13]:
ruta_audio_entrada = AUDIO_FILES_PATH + FILENAME + EXTENCION_ENTRADA
ruta_audio_entrada_convertido = AUDIO_FILES_PATH + 'waw_conv_' + FILENAME + EXTENCION_SALIDA

In [14]:
def record_audio(audio_file_path = AUDIO_FILES_PATH, filename = FILENAME, duration = DURATION, sample_rate = SAMPLE_RATE):
    print("Grabando...")
    # Grabar audio con 1 canal (mono)
    recording = sd.rec(int(duration * sample_rate), samplerate=sample_rate, channels=1)
    sd.wait()  # Esperar a que termine la grabación
    print("Grabación finalizada")
    write(ruta_audio_entrada, sample_rate, recording)
    return filename

In [15]:
def convert_to_wav(input_file = ruta_audio_entrada, output_file = ruta_audio_entrada_convertido):
    sound = AudioSegment.from_file(input_file)
    sound.export(output_file, format="aiff")

In [16]:
def chatbot():
    record_audio()

In [ ]:
chatbot()

In [ ]:
# Uso de la función convert_to_wav
convert_to_wav(ruta_audio_entrada, ruta_audio_entrada_convertido)

#### Reconocimiento del Habla (ASR - Automatic Speech Recognition):

In [ ]:
# Crear un objeto Recognizer
recognizer = sr.Recognizer()

In [ ]:
def recognize_speech():
    recognizer = sr.Recognizer()

    try:
        with sr.AudioFile(ruta_audio_entrada_convertido) as source:
            audio_data = recognizer.record(source)
            text = recognizer.recognize_sphinx(audio_data, language="es-ES")
            print("Texto reconocido:", text)
    except sr.UnknownValueError:
        print("No se pudo entender el audio")
    except sr.RequestError as e:
        print("Error al solicitar resultados de reconocimiento de voz; {0}".format(e))

In [ ]:
# Ejecutar la función para reconocer el discurso
recognize_speech()

In [ ]:
# Abrir el archivo de audio
def transcribe_audio(audio_lang = 'es-ES'):
    with sr.AudioFile(ruta_audio_entrada_convertido) as source:
        # Escuchar el audio (en inglés)
        audio = recognizer.record(source)

        # Utilizar Google Speech Recognition para transcribir el audio
        try:
            text = recognizer.recognize_google(audio, audio_lang)
            print("Transcripción: ", text)
            return text
        except sr.UnknownValueError:
            print("No se pudo entender el audio")
        except sr.RequestError as e:
            print("Error al solicitar resultados del servicio Google Speech Recognition; {0}".format(e))

In [ ]:
transcribe_audio()

In [ ]:
# Transcribir y mostrar el resultado
text = transcribe_audio(audio_file_path, audio_lang)

## **2. Procesamiento del Lenguaje Natural (NLP)**

In [17]:
!pip install spacy

In [18]:
# Descargar el modelo de spaCy para español desde un Jupyter Notebook
!python3 -m spacy download es_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 1.4 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')


In [19]:
import spacy
import re

In [20]:
# Cargar el modelo de spaCy para español
nlp = spacy.load("es_core_news_sm")

### **Preprocesamiento de Texto:**


In [21]:
# Lista de abreviaturas y formas no abreviadas de unidades de medida de peso
unidades_medida_peso = ['g', 'gr', 'kg', 'mg', 'µg', 't', 'lb', 'oz', 'cwt',
                        'gramos', 'kilogramos', 'miligramos', 'microgramos',
                        'toneladas', 'libras', 'onzas', 'quintales']

In [25]:
def procesar_texto(texto):
    texto = texto.lower()
    texto = re.sub(r'[,\.;:!\-*#@$!+_%^&`~]', '', texto)
    texto = re.sub(r'\s+', ' ', texto)

    thresgold = 1

    doc = nlp(texto)

    tokens = [token.text for token in doc]
    stopwords = spacy.lang.es.stop_words.STOP_WORDS
    filtered_tokens = [token.text for token in doc if token.text not in stopwords and token.pos_ in ('NOUN', 'ADJ', 'ADP') and len(token.text) > thresgold and token.text not in unidades_medida_peso]
    lemmas = [token.lemma_ for token in doc if token.text in filtered_tokens]
    pos_tags = [(token.text, token.pos_) for token in doc if token.text in filtered_tokens]

    return {
        "tokens": tokens,
        "lemmas": lemmas,
        "filtered_tokens": filtered_tokens,
        "pos_tags": pos_tags,
    }


**Normalización del texto (eliminación de ruido, corrección ortográfica, etc.)**

In [26]:
texto_analizado = procesar_texto(texto_prueba)

**Tokenización**

In [27]:
tokens = texto_analizado['tokens']

In [28]:
tokens

['estoy',
 'buscando',
 'tomates',
 'en',
 'oferta',
 'pero',
 'que',
 'esten',
 'bien',
 'rojitos',
 'p']

**Eliminación de stop words.**


In [29]:
tokens_filtrados = texto_analizado['filtered_tokens']

In [30]:
tokens_filtrados

['tomates', 'oferta', 'rojitos']

**Lematización y stemming**

In [31]:
lemmas_filtrados = texto_analizado['lemmas']

In [32]:
lemmas_filtrados

['tomate', 'oferta', 'rojito']

Etiquetado de estructuras gramaticales:

In [33]:
pos_tags_filtrados = texto_analizado['pos_tags']

In [34]:
pos_tags_filtrados

[('tomates', 'NOUN'), ('oferta', 'NOUN'), ('rojitos', 'ADJ')]

### **Análisis de Texto:**


Detectar presencia de palabras de productos lematizadas dentro de dataframe

In [35]:
df_frutas_verduras['producto_tokens_lemmas'] = df_frutas_verduras['Producto'].apply(lambda x: procesar_texto(x)['lemmas'])

In [52]:
df_frutas_verduras['producto_tokens_lemmas']

0          [suelto]
1          [perita]
2                []
3           [atado]
4         [paquete]
           ...     
456         [marca]
457    [energetico]
458        [school]
459    [patagonico]
460      [tropical]
Name: producto_tokens_lemmas, Length: 461, dtype: object

In [39]:
lista = [(index, row) for index, row in df_frutas_verduras.iterrows()]

In [50]:
type(lista[1][1])

pandas.core.series.Series

In [41]:
dict_count_coincidences = {}

In [ ]:
for index, row in df_frutas_verduras.iterrows():
    for token_producto in row['producto_tokens_lemmas']:
        if token_producto in lemmas_filtrados:
            if token_producto not in dict_count_coincidences:
                dict_count_coincidences[index] = 1
            else:
                dict_count_coincidences[index] += 1

In [72]:
series_ordered_count_coincidences = pd.Series(dict_count_coincidences).sort_values

In [73]:
series_ordered_count_coincidences

<bound method Series.sort_values of 29     1
76     1
89     1
182    1
271    1
296    1
308    1
dtype: int64>

Análisis de Sentimiento: Determinar la emoción o tono del texto.


In [ ]:
#para futuras versiones

Detección de Intenciones (Intent Detection): Identificar la intención del usuario utilizando modelos como BERT, GPT, RASA, etc.


In [ ]:
#para futuras versiones

## **3. Gestión del Diálogo**

### Módulo de Gestión de Estado:
Llevar un registro del contexto y estado del diálogo para mantener conversaciones coherentes.


### Motor de Respuesta:


Generación de Respuestas: Utilizar modelos generativos (como GPT-3) o respuestas predefinidas según las intenciones y entidades detectadas.


Selección de Respuestas: Elegir la mejor respuesta entre varias opciones generadas.


### Personalización y Contexto: Adaptar las respuestas en función del historial del usuario y el contexto actual.

## 4. Salida de Usuario (Text-to-Speech)


### Conversión de Texto a Voz (TTS): Utilizar servicios como Google Text-to-Speech, Amazon Polly, o frameworks como Tacotron para convertir el texto generado en voz.


### Reproducción de Audio: Entregar la respuesta de voz al usuario.